<h1><b>Diamond price prediciton</b></h1>

<h1><b>Libs Importing</b></h1>

In [1]:
# Install AutoGluon.
# AutoGluon is used in AutoML.

!pip install -U pip
!pip install -U setuptools wheel

!pip install torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cpu

!pip install autogluon

  Using cached setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
Using cached setuptools-69.5.1-py3-none-any.whl (894 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 60.2.0
    Uninstalling setuptools-60.2.0:
      Successfully uninstalled setuptools-60.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
openxlab 0.0.38 requires setuptools~=60.2.0, but you have setuptools 69.5.1 which is incompatible.


Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached setuptools-60.2.0-py3-none-any.whl.metadata (5.1 kB)
Using cached setuptools-60.2.0-py3-none-any.whl (953 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.5.1
    Uninstalling setuptools-69.5.1:
      Successfully uninstalled setuptools-69.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
cvxpy 1.3.4 requires setuptools>65.5.1, but you have setuptools 60.2.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<h1><b>Loading Data</b></h1>

In [3]:
# Load the data
df = pd.read_csv('train.csv')
df.head()

,Id,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,1.06,Ideal,I,SI2,61.8,57.0,4270,6.57,6.60,4.07
1,2,1.51,Premium,G,VVS2,60.9,58.0,15164,7.38,7.42,4.51
2,3,0.32,Ideal,F,VS2,61.3,56.0,828,4.43,4.41,2.71
3,4,0.53,Ideal,G,VS2,61.2,56.0,1577,5.19,5.22,3.19
4,5,0.70,Premium,H,VVS2,61.0,57.0,2596,5.76,5.72,3.50


In [4]:
df.drop("Id",inplace=True,axis=1)
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.06,Ideal,I,SI2,61.8,57.0,4270,6.57,6.60,4.07
1,1.51,Premium,G,VVS2,60.9,58.0,15164,7.38,7.42,4.51
2,0.32,Ideal,F,VS2,61.3,56.0,828,4.43,4.41,2.71
3,0.53,Ideal,G,VS2,61.2,56.0,1577,5.19,5.22,3.19
4,0.70,Premium,H,VVS2,61.0,57.0,2596,5.76,5.72,3.50


In [5]:
Cut_dictionary = {'Ideal':5, 'Premium':4, 'Very Good':3, 'Fair':1, 'Good':2}
Color_dictionary = {'J':1, 'I':2, 'H':3, 'G':4, 'F':5, 'E':6, 'D':7}
Clarity_dictionary = {'I1':1, 'SI2':2, 'SI1':3, 'VS2':4, 'VS1':5, 'VVS2':6, 'VVS1':7, 'IF':8}

In [6]:
def encode(data):
    df =data.copy()
    df["cut"] = df["cut"].map(Cut_dictionary)
    df["color"] = df["color"].map(Color_dictionary)
    df["clarity"] = df["clarity"].map(Clarity_dictionary)
    return df

In [7]:
df = encode(df)

<h1><b>Feature Engineering</b></h1>

In [8]:
#drop null values
df = df.dropna()

# Create new features
df['volume'] = df['x'] * df['y'] * df['z']
df['volume_depth_ratio'] = df['volume'] / df['depth']
df['volume_table_ratio'] = df['volume'] / df['table']

#Remove the uncorrelated features with price
df.drop(['depth', 'cut'], axis = 1, inplace = True)

<h1><b>Data Preprocessing<b></h1>

In [9]:
#split data before applying Data Processing
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

def oversample_feature(train_df, feature, desired_percentage):
    # Split the training data into features and target variable
    X_train = train_df.drop(columns=['price'])
    y_train = train_df['price']

    # Calculate the total number of samples
    total_samples = len(train_df)

    # Calculate the desired count for each unique value based on the desired percentage
    desired_count_per_value = int(total_samples * desired_percentage)

    # Create a list containing each unique value in the feature column
    unique_values = sorted(X_train[feature].unique())

    # Create a dictionary to store the number of occurrences of each value
    value_counts = {value: 0 for value in unique_values}

    # Iterate over each row in the dataframe and update the value counts
    for value in X_train[feature]:
        value_counts[value] += 1

    # Oversample or undersample the dataframe to match the desired count for each value
    oversampled_dfs = []
    for value in unique_values:
        value_df = train_df[train_df[feature] == value]
        current_count = value_counts[value]
        if current_count < desired_count_per_value:
            # Oversample if the count is less than the desired count
            oversampled_df = value_df.sample(n=desired_count_per_value, replace=True, random_state=42)
        elif current_count > desired_count_per_value:
            # Undersample if the count is more than the desired count
            oversampled_df = value_df.sample(n=desired_count_per_value, random_state=42)
        else:
            oversampled_df = value_df.copy()
        oversampled_dfs.append(oversampled_df)

    # Concatenate the oversampled dataframes
    train_df_resampled = pd.concat(oversampled_dfs)

    return train_df_resampled

# Apply oversampling for the 'color' feature to the training data
train = oversample_feature(train, 'color', 0.1428)

# Apply oversampling for the 'clarity' feature to the training data
train = oversample_feature(train, 'clarity', 0.125)

In [11]:
#Applying logarithmic scale to the price to solve the problem of highly left skewed data and small the range of the prices
train['price'] = np.log1p(train['price'])
test['price'] = np.log1p(test['price'])

In [12]:
#applying logarithmic transformation and normalization to train and test features

from sklearn.preprocessing import StandardScaler
import numpy as np

# Log-transform features (excluding 'price')
log_features = df.drop(['price','color', 'clarity'], axis=1).columns
for feature in log_features:
    train[feature] = np.log1p(train[feature])
    test[feature] = np.log1p(test[feature])

# Separate 'price' column from the features
train_price = train['price']
test_price = test['price']
train_color = train['color']
test_color = test['color']
train_clarity = train['clarity']
test_clarity = test['clarity']

# Select features for normalization (excluding 'price')
features_for_normalization = train.drop(['price','color', 'clarity'], axis=1).columns

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to your data
scaler.fit(train[features_for_normalization])

# Transform the data using the scaler
normalized_train_features = scaler.transform(train[features_for_normalization])
normalized_test_features = scaler.transform(test[features_for_normalization])

# Convert the scaled data back to a DataFrame
normalized_train = pd.DataFrame(normalized_train_features, columns=features_for_normalization)
normalized_test = pd.DataFrame(normalized_test_features, columns=features_for_normalization)

# Reset the index of train_price and test_price
train_price = train_price.reset_index(drop=True)
test_price = test_price.reset_index(drop=True)

# Add 'price, color and clarity' columns back to the DataFrame
train_color = train_color.reset_index(drop=True)
test_color = test_color.reset_index(drop=True)
train_clarity = train_clarity.reset_index(drop=True)
test_clarity = test_clarity.reset_index(drop=True)

normalized_train['price'] = train_price
normalized_test['price'] = test_price
normalized_train['color'] = train_color
normalized_test['color'] = test_color
normalized_train['clarity'] = train_clarity
normalized_test['clarity'] = test_clarity

#change type of 'color' and 'clarity' columns
normalized_train['color'] = normalized_train['color'].astype('category')
normalized_test['color'] = normalized_test['color'].astype('category')
normalized_train['clarity'] = normalized_train['clarity'].astype('category')
normalized_test['clarity'] = normalized_test['clarity'].astype('category')

**Train model**

In [13]:
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor(label = 'price', eval_metric = 'root_mean_squared_error').fit(train_data = normalized_train, time_limit = 2000, presets = 'best_quality');

No path specified. Models will be saved in: "AutogluonModels/ag-20240521_081822"
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 2000 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240521_081822/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 705 second

**Evaluate model**

In [14]:
from sklearn.metrics import r2_score

predictions   = predictor.predict(normalized_test)
actual_values = normalized_test['price']

# Take the inverse of the logarithmic transformation for predicted prices
predictions_exp = np.exp(predictions)

# Take the inverse of the logarithmic transformation for actual prices
actual_values_exp = np.exp(actual_values)

r2 = r2_score(actual_values_exp, predictions_exp)

r2

0.9812931672741864

In [15]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Calculate Mean Squared Error
mse = mean_squared_error(actual_values_exp, predictions_exp)

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

rmse

538.1464950308223